In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pickle


In [2]:
#Import training data yg dah cleaned
df = pd.read_csv('/Users/faiz/FYPCoding/FYP/newtrainingdata.csv')


In [3]:
# df[df.sentiment == 1] #798478
# dfneg = df.iloc[0:20000]
# dfpos = df.iloc[798478:818478]
# df = pd.concat([dfneg, dfpos], ignore_index=True)
#Positive and negative sentiment count
print(df.sentiment.value_counts())
#Percentage of positive and negative reviews
print(df.sentiment.value_counts() / len(df))
#df.text.str.len()

1    798679
0    798479
Name: sentiment, dtype: int64
1    0.500063
0    0.499937
Name: sentiment, dtype: float64


In [4]:
#####CountVectorizer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS 

#to add new stop word
#my_stop_words = ENGLISH_STOP_WORDS.union(['airline', 'airlines', '@'])

# Build the vectorizer, specify max features 
cv = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b[^\d\W][^\d\W]+\b',
                     stop_words=ENGLISH_STOP_WORDS).fit(df.text)

# Transform the review column
cv_transform = cv.transform(df.text)

####SAVE BagOfWords model
# save the model to disk
filename = 'cv_fitted.sav'
pickle.dump(cv, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

# Create the bow representation
# df_cv=pd.DataFrame(cv_transform.toarray(), columns=cv.get_feature_names())
# print(df_cv.head())

In [5]:
####TFIDF 
from sklearn.feature_extraction.text import TfidfVectorizer

# Define the vectorizer and specify the arguments
my_pattern = r'\b[^\d\W][^\d\W]+\b'
tvc = TfidfVectorizer(ngram_range=(1, 2),token_pattern=my_pattern, 
                      stop_words=ENGLISH_STOP_WORDS,sublinear_tf=True).fit(df.text)

# Transform the vectorizer
tvc_transform = tvc.transform(df.text)

####SAVE BagOfWords model
# save the model to disk
filename = 'tvc_fitted.sav'
pickle.dump(tvc, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))


# Transform to a data frame and specify the column names
###nk convert ke df kne upgrade ram
# df_tvc=pd.DataFrame(tvc_transform.toarray(), columns=tvc.get_feature_names())
# print('Top 5 rows of the DataFrame: ', df_tvc.head())

In [6]:
# Import the required packages
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix

# Define the vector of labels and matrix of features FOR TFIDF
y_tvc = df.sentiment
X_tvc = tvc_transform
# Define the vector of labels and matrix of features FOR COUNTVECTORIZER
y_cv = df.sentiment
X_cv = cv_transform

# Perform the train-test split FOR TFIDF
X_train_tvc, X_test_tvc, y_train_tvc, y_test_tvc = train_test_split(X_tvc, y_tvc, test_size=0.2, random_state=42,stratify=y_tvc)
# Perform the train-test split FOR COUNTVECTORIZER
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_cv, y_cv, test_size=0.2, random_state=42,stratify=y_cv)

# Build a classification model FOR TFIDF
model_tvc = MultinomialNB().fit(X_train_tvc,y_train_tvc)
# Build a classification model FOR COUNTVECTORIZER
model_cv = MultinomialNB().fit(X_train_cv,y_train_cv)
#multinb = MultinomialNB().fit(X_testtvc,y_testtvc)

print('TF-IDF Performance')
print('Accuracy on train set: ', model_tvc.score(X_train_tvc,y_train_tvc))
print('Accuracy on test set: ', model_tvc.score(X_test_tvc,y_test_tvc))
y_predicted_tvc = model_tvc.predict(X_test_tvc)
print('Confusion matrix test set: \n', confusion_matrix(y_test_tvc, y_predicted_tvc)/len(y_test_tvc))
print()
print()
print('Count Vectorizer Performance')
print('Accuracy on train set: ', model_cv.score(X_train_cv,y_train_cv))
print('Accuracy on test set: ', model_cv.score(X_test_cv,y_test_cv))
y_predicted_cv = model_cv.predict(X_test_cv)
print('Confusion matrix test set: \n', confusion_matrix(y_test_cv, y_predicted_cv)/len(y_test_cv))





# # Make predictions on the test set
# y_predicted = model.predict(X_test)

# # Predict the probability of the 0 class
# prob_0 = log_reg.predict_proba(X_test)[:, 0]
# # Predict the probability of the 1 class
# prob_1 = log_reg.predict_proba(X_test)[:, 1]

# print("First 10 predicted probabilities of class 0: ", prob_0[:10])
# print("First 10 predicted probabilities of class 1: ", prob_1[:10])
# # Print the performance metrics


# #####FOR REGULARIZATION
# # Train a logistic regression with regularization of 1000
# log_reg1 = LogisticRegression(C=1000).fit(X_train, y_train)
# # Train a logistic regression with regularization of 0.001
# log_reg2 = LogisticRegression(C=0.001).fit(X_train, y_train)



TF-IDF Performance
Accuracy on train set:  0.8918531829202818
Accuracy on test set:  0.7752103734128077
Confusion matrix test set: 
 [[0.39361116 0.10632623]
 [0.1184634  0.38159921]]


Count Vectorizer Performance
Accuracy on train set:  0.8963533652754972
Accuracy on test set:  0.7741553757920309
Confusion matrix test set: 
 [[0.39273147 0.10720592]
 [0.11863871 0.3814239 ]]


In [8]:
# save the model to disk
filename = 'tvcsent_model.sav'
pickle.dump(model_tvc, open(filename, 'wb'))

# save the model to disk
filename = 'cvsent_model.sav'
pickle.dump(model_cv, open(filename, 'wb'))
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test_cv, y_test_cv)
print(result)

0.7741553757920309


In [ ]:
###tvc = TfidfVectorizer(token_pattern=my_pattern, stop_words=ENGLISH_STOP_WORDS,sublinear_tf=True).fit(df.text)

Accuracy on train set:  0.7889555350677688
Accuracy on test set:  0.7573255027674122
Accuracy score test set:  0.7573255027674122
Confusion matrix test set: 
 [[0.38130807 0.11862932]
 [0.12404518 0.37601743]]

#tvc = TfidfVectorizer(ngram_range=(1, 2),token_pattern=my_pattern, stop_words=ENGLISH_STOP_WORDS,sublinear_tf=True).fit(df.text)
0.7752103734128077
Accuracy on train set:  0.8918531829202818
Accuracy on test set:  0.7752103734128077
Accuracy score test set:  0.7752103734128077
Confusion matrix test set: 
 [[0.39361116 0.10632623]
 [0.1184634  0.38159921]]

In [ ]:
###cv = CountVectorizer(token_pattern=r'\b[^\d\W][^\d\W]+\b',stop_words=ENGLISH_STOP_WORDS)
Accuracy on train set:  0.7895315584092364
Accuracy on test set:  0.7637431440807433
Accuracy score test set:  0.7637431440807433
Confusion matrix test set: 
 [[0.38525257 0.11468482]
 [0.12157204 0.37849057]]


#cv = CountVectorizer(ngram_range=(1, 2),token_pattern=r'\b[^\d\W][^\d\W]+\b',stop_words=ENGLISH_STOP_WORDS)
0.7741553757920309
Accuracy on train set:  0.8963533652754972
Accuracy on test set:  0.7741553757920309
Accuracy score test set:  0.7741553757920309
Confusion matrix test set: 
 [[0.39273147 0.10720592]
 [0.11863871 0.3814239 ]]